In [ ]:
!pip install underthesea
!git clone https://github.com/stopwords/vietnamese-stopwords

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 6.6 MB/s 
     |████████████████████████████████| 965 kB 72.5 MB/s 
     |████████████████████████████████| 235 kB 71.6 MB/s 
     |████████████████████████████████| 581 kB 78.2 MB/s 
Cloning into 'vietnamese-stopwords'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 88 (delta 1), reused 0 (delta 0), pack-reused 81
Unpacking objects: 100% (88/88), done.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import underthesea

In [ ]:
from unidecode import unidecode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/PhoBert/archive.zip

Archive:  /content/drive/MyDrive/PhoBert/archive.zip
  inflating: data - data.csv         
  inflating: data.csv                


In [ ]:
df = pd.read_csv('/content/data.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PhoBert/commnet.csv',encoding= 'unicode_escape')

UnicodeDecodeError: ignored

In [ ]:
df = df[(df['label'] == 'POS') | (df['label'] == 'NEG')]

**SOME FUNCTION PREPROCESSING**

In [ ]:
stopwords_path = "/content/vietnamese-stopwords/vietnamese-stopwords.txt"

def standardize_data(sentence):
    row = re.sub(r"[\.,\?]+$-","",sentence)
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    row = row.strip().lower()
    return row

def cleaning_text(text, keep_punct=True):
    if keep_punct:
        return re.sub(u'[^{Latin}0-9[:punct:]]+', u' ', text)
    return re.sub(u'[^{Latin}0-9]+', u' ', text)

def load_stopwords():
    sw = []
    with open(stopwords_path, encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        sw.append(line.replace("\n",""))
    return sw


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

sw = load_stopwords()
def preprocessing(text, keep_punct=True):
    text = standardize_data(text)
    text = remove_emoji(text)
    #tokenize
    line = underthesea.word_tokenize(text)

    #filter stopword
    filter_words = [word for word in line if word not in sw]
    #concat
    line = "".join(text)
    line = underthesea.word_tokenize(line,format = "text")
    
    return cleaning_text(line)

In [ ]:
df = df.dropna()
df.content = df.content.map(lambda x : preprocessing(x))

In [ ]:
df.to_csv("/content/drive/MyDrive/PhoBert/clean_data.csv")

In [ ]:
df

,content,label,start
0,áo_bao đẹp ạ,POS,5
1,tuyệt_vời,POS,5
2,2 day ao khong giong trong,NEG,1
3,mùi thơm bôi lên_da mềm da,POS,5
4,vải đẹp dày_dặn,POS,5
...,...,...,...
31455,không đáng tiền,NEG,1
31456,quần rất đẹp,POS,5
31457,hàng đẹp đúng giá tiền,POS,5
31458,chất vải khá ổn,POS,4
